# Example hologram processing pipeline

### Step 00 - configure warnings and autoreload

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

### Step 01 - Imports

In [ ]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import os

from pyopia.classify import Classify
import pyopia.process
import pyopia.io
import pyopia.background
import pyopia.statistics
import exampledata
from pyopia.pipeline import Pipeline
import pyopia.instrument.holo as holo

#download the default holo test data and create file list
foldername = exampledata.get_folder_from_holo_repository("holo_test_data_01")


In [ ]:
# download the holo test data
# either holo_test_data_01 (default) or holo_test_data_02
infolder = exampledata.get_folder_from_holo_repository("holo_test_data_02")

### Step 03 - Define runtime settings

In [ ]:
# path to folder of input data
infolder = 'holo_test_data_01'

# path to folder of output data (will be created)
outfolder = 'proc1'

# name of output statistics file (if already exists will be removed)
outfilename = 'holotest'

# path to classifier model to use (here we download and use an example model)
model_path = exampledata.get_example_model()

# segmentation threshold 
threshold = 0.9

# number of images to use as background (recommend >=10) - for either static or moving background removal
average_window = 10  

# hologram reconstruction settings
holo_initial_settings = {'pixel_size': 4.4, # pixel size in um
                        'wavelength': 658, # laser wavelength in nm
                        'n': 1.33, # index of refraction of sample volume medium (1.33 for water)
                        'offset': 26, # offset to start of sample volume in mm
                        'minZ': 0, # minimum reconstruction distance within sample volume in mm
                        'maxZ': 50, # maximum reconstruction distance within sample volume in mm
                        'stepZ': 0.5} #step size in mm

### Step 04 - Setup folders & input file lists

In [ ]:
# make output folder
os.makedirs(outfolder, exist_ok=True)

# remove pre-existing output file (as statistics for each image are appended to it)
datafile_hdf = os.path.join(outfolder,'holotest')
if os.path.exists(datafile_hdf + '-STATS.h5'):
  os.remove(datafile_hdf + '-STATS.h5')

# get sorted list of input files
files = sorted(glob(os.path.join(infolder, '*.pgm')))

# create list of files for initial creation of background file
bgfiles = files[:average_window]

### Step 05 - Setup pipeline steps

In [ ]:
# define the steps to use in the processing pipeline - given as a label and the Class with options
steps = {
         'initial': holo.Initial(files[0], **holo_initial_settings), # initial step, run once on pipeline initialisation - arguments a filename to use for sizing the image and hologram reconstruction settings from Step 03
         'classifier': Classify(model_path=model_path), # sets up classifier model, run once on pipeline initialisation - argument is the path to the classification model to use from Step 03
         'create background': pyopia.background.CreateBackground(bgfiles, pyopia.instrument.holo.load_image),
         'load': holo.Load(),
         'correct background': pyopia.background.CorrectBackgroundAccurate(pyopia.background.shift_bgstack_accurate),
         'reconstruct': holo.Reconstruct(stack_clean=0.02),
         'focus': holo.Focus(pyopia.instrument.holo.std_map,threshold=threshold,focus_function=pyopia.instrument.holo.find_focus_sobel),
         'segmentation': pyopia.process.Segment(threshold=threshold),
         'statextract': pyopia.process.CalculateStats(export_outputpath=outfolder),
         'merge holo stats': holo.MergeStats(),
         'output': pyopia.io.StatsH5(datafile_hdf)
         }

# now initialise the pipeline
processing_pipeline = Pipeline(steps)

In [ ]:
#do a test run with just the first few input files
for filename in files[:5]:
    stats = processing_pipeline.run(filename)

In [ ]:
# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

In [ ]:
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))
# Calculate the volume distribution from the stats DataFrame.
dias, vd = pyopia.statistics.vd_from_stats(stats, 24)

# plot the volume distribution
plt.style.use('dark_background')
plt.plot(dias, vd)
plt.xscale('log')
plt.xlabel('ECD [um]')
plt.ylabel('Volume Distribution [uL/sample vol.]')

In [ ]:
#plot histogram of focus locations
import numpy as np
plt.style.use('dark_background')
zval = np.arange(holo_initial_settings['minZ'], holo_initial_settings['maxZ'], holo_initial_settings['stepZ'])
plt.hist(zval[stats.ifocus-1],len(zval))
plt.xlim(zval[0],zval[-1])
plt.xlabel('Z [mm]')

In [ ]:
im_mont = pyopia.statistics.make_montage(datafile_hdf + '-STATS.h5',holo_initial_settings["pixel_size"],outfolder,
    auto_scaler=1000, msize=2048, maxlength=100000, crop_stats=None)
pyopia.statistics.montage_plot(im_mont,holo_initial_settings['pixel_size'])